# 1. Pytorch → onnx

In [2]:
import os

os.chdir('../')


In [3]:
import torch

from latency_predictor.arch_utils import *

from nas.supernet.supernet_yolov7 import YOLOSuperNet

device = 'cpu'
supernet = YOLOSuperNet(cfg='yaml_config/yolov7_supernet.yml').to(device)

supernet.set_active_subnet([5,5,5,5,7,7,7,1])
subnet = supernet.get_active_subnet()
b_arch, h_arch, save = split_backbone_head(subnet)
backbone = skin_backbone(b_arch, save).eval()
head = skin_head(h_arch, save).eval()

img = torch.rand(1, 3, 640, 640).to(device)
torch.onnx.export(backbone,
                  img,
                  'exported_models/backbone_5555.onnx',
                  export_params=True,
                  verbose=False,
                  opset_version=10,        
                  do_constant_folding=False,
                  input_names = ['input'],
                  output_names = ['output'])

img = torch.rand(1, 1024, 20, 20).to(device)
torch.onnx.export(head,
                  img,
                  'exported_models/head_7771.onnx',
                  export_params=True,
                  verbose=False,
                  opset_version=10,
                  do_constant_folding=False,
                  input_names = ['input'],
                  output_names = ['output'])

/userHome/userhome1/hanyong/yolov7-nas-for-TANGO/nas/supernet/dynamic_layers/dynamic_op.py:246: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if bn.num_features == feature_dim or DynamicBatchNorm2d.SET_RUNNING_STATISTICS:
/userHome/userhome1/hanyong/miniconda3/envs/openvivo/lib/python3.7/site-packages/torch/onnx/symbolic_helper.py:821: UserWarning: You are trying to export the model with onnx:Resize for ONNX opset version 10. This operator might cause results to not match the expected results by PyTorch.
ONNX's Upsample/Resize operator did not match Pytorch's Interpolation until opset 11. Attributes to determine how to transform the input were added in onnx:Resize in opset 11 to support Pytorch's behavior (like coordinate_transformation_mode and nearest_mode).
We recom

# 2. onnx → onnx-simplify

In [5]:
import onnx
from onnxsim import simplify

onnx_model_paths = [
    'exported_models/backbone_5555.onnx',
    'exported_models/head_7771.onnx'
]

for path in onnx_model_paths:
    onnx_model = onnx.load(path)

    # convert model
    model_simp, check = simplify(onnx_model)

    assert check, "Simplified ONNX model could not be validated"
    onnx.save(model_simp, path)

# 3. onnx-simplify → OpenVINO

In [6]:
!mo --input_model "../exported_models/head_7771.onnx" --compress_to_fp16 --output_dir "../exported_models/openvino"

[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression by removing argument --compress_to_fp16 or set it to false --compress_to_fp16=False.
Find more information about compression to FP16 at https://docs.openvino.ai/latest/openvino_docs_MO_DG_FP16_Compression.html
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/latest/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /userHome/userhome1/hanyong/yolov7-nas-for-TANGO/exported_models/openvino/head_7_7_7_1.xml
[ SUCCESS ] BIN file: /userHome/userhome1/hanyong/yolov7-nas-for-TANGO/exported_models/openvino/hea

# 4. OpenVINO → tflite

In [ ]:
docker pull ghcr.io/pinto0309/openvino2tensorflow:latest

docker run -it --rm \
  -v `pwd`:/home/user/workdir \
  ghcr.io/pinto0309/openvino2tensorflow:latest

MODEL_NAME=head_7771
MODEL_PATH=exported_models/openvino/${MODEL_NAME}.xml
OUTPUT_PATH=exported_models/openvino/tflite/${MODEL_NAME}

sudo openvino2tensorflow --model_path $MODEL_PATH --model_output_path $OUTPUT_PATH --output_saved_model --output_float16_quant_tflite

sudo mv ${OUTPUT_PATH}/model_float16_quant.tflite ${OUTPUT_PATH}_f16.tflite
sudo rm -rf $OUTPUT_PATH